# Data Augmentation + CNN 모델 만들기
Test Accuracy 80% 이상으로 만들기!
## 알게 된 점
- 레이어 깊이가 적당히 작을수록 좋다
- optimizer = Adam이 좋다
- dropout 비율을 키우고 Conv2D 의 filter 크기 줄이고 Dense 인풋 크기 줄이면 overfitting 줄여준다.
- overfitting 줄여주어 결과적으로 train acc 와 test acc 랑 비슷하게 증가하게 된다.
- 반면 train set에서 acc가 더디게 증가할 수 있다.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras import applications
from keras import Model

print(tf.__version__)

# MNIST 데이터를 불러옵니다.
mnist = np.load('mnist.npz')
# TODO : Train Data와 Test Data로 분리해보세요.
# 데이터의 개수는 Train 100개, Test 20개로 분리합니다.
X_train, X_test, y_train, y_test = mnist['x_train'][:100], mnist['x_test'][:20], mnist['y_train'][:100], mnist['y_test'][:20]

# TODO : Train Data의 Pixel 값을 0 ~ 255에서 0 ~ 1 사이의 Float 데이터로 바꿔보세요.
X_train = X_train.astype(np.float)/255.
X_test = X_test.astype(np.float)/255.

# TODO : (num of data, 28, 28) 형태의 데이터를 (num of data, 28, 28, 1)로 만들어보세요. 
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Data Augmenation을 위한 메서드입니다.
def data_augmentation(image, label):
    rotate_img = []
    rotate_label = []
    flip_img = []
    flip_label = []
    
    print('Doing image argumentation...\n')
    for x, y in zip(image, label):
        # TODO : 입력 영상을 Rotate 해보세요. 단 Rotate 각도는 Random으로 설정해주세요.
        rotate_img.append(tf.image.rot90(x,int(np.random.randint(4,size=1))))
        rotate_label.append(y)
        
        # TODO : 입력 영상을 좌우반전해보세요.
        flip_img.append(tf.image.random_flip_left_right(x,0))
        flip_label.append(y)
        
        
    aug_img = np.array(rotate_img + flip_img)
    aug_label = np.array(rotate_label + flip_label)
    print('Image argumentation DONE...\n')
    
    return aug_img, aug_label
    
# TODO : Augmenation을 거친 데이터를 저장해주세요.
aug_X_train, aug_y_train = data_augmentation(X_train, y_train)

print('Doing Concatenate...\n')
X_train = np.concatenate((X_train,aug_X_train))
y_train = np.concatenate((y_train,aug_y_train))

# TODO : 0 ~ 9값을 가진 Data Label을 One Hot encoding 해주세요
y_train = to_categorical(y_train, 10)    
y_test = to_categorical(y_test, 10)    

# TODO : CNN 모델을 만들어보세요.
def CNN():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10))
    return model

# 모델이 어떻게 생겼는지 확인해보세요.
model = CNN()
model.summary()

# TODO : 모델을 학습할 방법과 Error 계산 방법, 평가 방법을 설정해보세요.

# 모델을 학습할 방법과 Error 계산 방법, 평가 방법을 설정합니다.
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

# 모델을 학습시켜줍니다.
# verbose의 값에 따라 출력 형태를 바꿀 수 있습니다.
# 0 : silent, 1 : progress bar, 2 : one line per epoch
history = model.fit(X_train, y_train, epochs = 45, batch_size = 64, validation_data = (X_test, y_test), verbose = 2)

# 테스트 데이터로 모델을 검증합니다.
loss, test_acc = model.evaluate(X_test, y_test)

# 결과 출력
print('Test Loss : {:.4f} | Test Accuracy : {}'.format(loss, np.round(test_acc,3)))
print('Test Data로 예측한 클래스 : ',model.predict_classes(X_test))
print('Test Data의 실제 클래스   : ',np.array([np.argmax(i) for i in y_test]))



2.0.0-beta1
Doing image argumentation...

Image argumentation DONE...

Doing Concatenate...

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 32)          0         
___________________________